In [1]:
import numpy as np
import pyaudio
import time
import librosa
import matplotlib.pyplot as plt
import librosa.display

from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.metrics import confusion_matrix
from PIL import Image, ImageFont, ImageDraw

In [ ]:
class AudioHandler(object):
    def __init__(self):
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 44100
        self.CHUNK = 1024 * 2
        self.RECORD_SECONDS = 1
        self.INPUT_DEVICE_INDEX = 1 # change this parametr to good work
        self.p = None
        self.stream = None

    def start(self):
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  #stream_callback=self.callback,
                                  frames_per_buffer=self.CHUNK,
                                  input_device_index = self.INPUT_DEVICE_INDEX)

    def stop(self):
        self.stream.close()
        self.p.terminate()

    def callback(self, in_data, frame_count, time_info, flag):
        numpy_array = np.frombuffer(in_data, dtype=np.float32)
        librosa.feature.mfcc(numpy_array)
        return None, pyaudio.paContinue

    def mainloop(self):
        while (self.stream.is_active()): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
            model = load_model(r'D:\save_1.h5')
            
            frames = []  
            for i in range(0, int(self.RATE / self.CHUNK * self.RECORD_SECONDS)):
                data = self.stream.read(self.CHUNK)
                frames.append(data)   
            frame = b''.join(frames)
            callback(self, frame, len(frames), )
            print('ok')
                
            time.sleep(2.0)


audio = AudioHandler()
audio.start()     # open the the stream
audio.mainloop()  # main operations with librosa
audio.stop()